## Stuttered focus image analysis

Craig Lage 16-Aug-23

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
from lsst_efd_client import EfdClient

import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
%matplotlib inline

from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask

In [ ]:
def plotExp(exp):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return ax1, figure

def repairCosmics(postIsr):
    # Borrowed from summit utils
    if postIsr.getPsf() is None:
        installPsfTask = InstallGaussianPsfTask()
        installPsfTask.run(postIsr)

    # TODO: try adding a reasonably wide Gaussian as a temp PSF
    # and then just running repairTask on its own without any
    # imChar. It should work, and be faster.
    repairConfig = CharacterizeImageTask.ConfigClass()
    repairConfig.doMeasurePsf = False
    repairConfig.doApCorr = False
    repairConfig.doDeblend = False
    repairConfig.doWrite = False
    repairConfig.repair.cosmicray.nCrPixelMax = 200000
    repairTask = CharacterizeImageTask(config=repairConfig)
    repairTask.repair.run(postIsr)
    return

def subtractStutteredBackground(exp, expcount = 20, rowshift = 100):
    for i in range(expcount):
        y0 = 2000 + rowshift * i
        y1 = 2000 + rowshift * (i+1)
        arr = exp.image.array[y0:y1,0:4071]
        median = np.median(arr)
        exp.image.array[y0:y1,0:4071] -= median
        y0 = 2000 - rowshift * i
        y1 = 2000 - rowshift * (i+1)
        arr = exp.image.array[y1:y0,0:4071]
        median = np.median(arr)
        exp.image.array[y1:y0,0:4071] -= median
    return 
 


In [ ]:
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib"])
client = EfdClient('usdf_efd')

expcount = 20
rowshift = 100


In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=True
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=True
isrConfig.doNanMasking=True
isrConfig.doInterpolate=True
isrConfig.doSaturation=True
isrConfig.doSaturationInterpolation=True
isrConfig.doWrite=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
expId = 2023081500058
exp = butler.get('raw', detector=0, exposure=expId)
biasExp = butler.get('bias', detector=0, exposure=expId)
flatExp = butler.get('flat', detector=0, exposure=expId)
darkExp = butler.get('dark', detector=0, exposure=expId)
defectExp = butler.get('defects', detector=0, exposure=expId)
isrResult = isrTask.run(exp, bias=biasExp, flat=flatExp, dark=darkExp, defects=defectExp)
postIsrExp = isrResult.exposure
repairCosmics(postIsrExp)
subtractStutteredBackground(postIsrExp)
ax1, fig = plotExp(postIsrExp)
ax1.set_title(f"Stuttered Focus Image {expId}", fontsize=24)
for i in range(expcount):
    if i > 2 and i % 2 == 1:
        ax1.arrow(2400,3950 - rowshift * (19 - i), -200, 0,\
            color='lightgreen', width = 20)
        ax1.text(2500,3925 - rowshift * (19 - i), f"Exposure_{int((19 - i) / 2) + 1}", color='lightgreen', fontsize=16)
    elif i > 2:
        ax1.arrow(1700,3950 - rowshift * (19 - i), 200, 0,\
            color='lightgreen', width = 20)
        ax1.text(1200,3925 - rowshift * (19 - i), "Shift focus", color='lightgreen', fontsize=16)
    else:
        ax1.arrow(1700,3950 - rowshift * (19 - i), 200, 0,\
            color='lightgreen', width = 20)
        ax1.text(1300,3925 - rowshift * (19 - i), "Dummy", color='lightgreen', fontsize=16)
plt.savefig(f"/home/c/cslage/u/AuxTel/stuttered/Stuttered_Focus_Image_{expId}.png")         

In [ ]:
# Set up the source catalog task
charConfig = CharacterizeImageConfig()
charConfig.doMeasurePsf = False
charConfig.doApCorr = False
charConfig.doDeblend = False
charConfig.repair.doCosmicRay = False
charConfig.repair.doInterpolate = False 
charConfig.detection.minPixels = 100
charTask = CharacterizeImageTask(config=charConfig)

In [ ]:
charResult = charTask.run(postIsrExp)
sourceCatalog = charResult.sourceCat


In [ ]:
mData = exp.getMetadata()
start = Time(mData['DATE-BEG'], format='isot', scale='tai') - TimeDelta(10.0, format = 'sec')
end = Time(mData['DATE-END'], format='isot', scale='tai') + TimeDelta(10.0, format = 'sec')

startCorrection = await client.select_time_series("lsst.sal.ATAOS.logevent_hexapodCorrectionStarted", \
                                                  "hexapod_z", start.utc, end.utc)
endCorrection = await client.select_time_series("lsst.sal.ATAOS.logevent_hexapodCorrectionCompleted", \
                                                  "hexapod_z", start.utc, end.utc)

In [ ]:
# Find the images corrresponding to the main star
x0 = 2000
x1 = 2100
numObjs = 0

xs = []
sizes = []
flux_xs = []
fluxes = []
for n, obj in enumerate(sourceCatalog):
    x = obj['base_SdssCentroid_x']
    y = obj['base_SdssCentroid_y']
    for i in range(expcount):
        y0 = 2000 + rowshift * i
        y1 = y0 + rowshift
        if x > x0 and x < x1 and y > y0 and y < y1:
            xx = obj['ext_shapeHSM_HsmSourceMoments_xx']
            yy = obj['ext_shapeHSM_HsmSourceMoments_yy']
            #print(xx,yy)
            size = np.sqrt(xx*xx + yy*yy)
            flux = obj['base_CircularApertureFlux_9_0_instFlux']
            flux_xs.append(19 - i)
            fluxes.append(flux)
            if i > 2 and i % 2 == 1:
                #print(i, int((19 - i) / 2) + 1)
                xs.append(int((19 - i) / 2) + 1)
                sizes.append(np.sqrt(size)/10.0 * 2.355)
            #print(i, n, x, y, size/10.0)
            numObjs += 1
print(numObjs)
fig, axs = plt.subplots(2,1,figsize=(8,8))
plt.suptitle(f"Stuttered Focus {expId}", fontsize=24)
plt.subplots_adjust(hspace=0.3)
axs[0].set_title("Flux vs shift number") 
axs[0].plot(flux_xs, fluxes)
axs[0].set_ylabel("Flux")
axs[0].set_xlabel("Shift number")
axs[0].text(8.0, 40000, "Clouds??", color='red')
axs[1].set_title("FWHM vs image number") 
axs[1].plot(xs, sizes)
axs[1].set_xlim(0,10)
axs[1].set_ylabel("FWHM(arcseconds)")
axs[1].set_xlabel("Image number")
axs[1].text(4, 3.5, f"Hex_z values, CWFS best={startCorrection['hexapod_z'].values[4]:.4f}")
#axs[1].text(7,3, f"CharacterizeImageTask\nfailed to find\nimage 9 ??")
for i in range(9):
    try:
        corr = startCorrection['hexapod_z'].values[8-i]
        axs[1].text(xs[i]+0.2, sizes[i], f"{corr:.4f}")
    except:
        continue
plt.savefig(f"/home/c/cslage/u/AuxTel/stuttered/Stuttered_Focus_Plot_{expId}.png") 

In [ ]:
startCorrection